# Qubitization Walk Operator

In [ ]:
import cirq
import numpy as np
import qualtran
import cirq_ft
import cirq_ft.infra.testing as cq_testing
from qualtran.jupyter_tools import display_gate_and_compilation, show_bloq
from typing import *

## `Walk`
Constructs a Szegedy Quantum Walk operator using `select` and `prepare`.

$\def\select{\mathrm{SELECT}} \def\prepare{\mathrm{PREPARE}} \def\r{\rangle} \def\l{\langle}$
Constructs the walk operator $W = R_L \cdot \select$, which is a product of
reflection $R_L = (2|L \r\l L| - I)$ and $\select=\sum_l |l\r\l l|H_l$.
$L$ is the state prepared by $\prepare|\vec0\r = |L\r$

The action of $W$ partitions the Hilbert space into a direct sum of two-dimensional irreducible
vector spaces. For an arbitrary eigenstate $|k\r$ of $H$ with eigenvalue $E_k$, $|\ell\r|k\r$
and an orthogonal state $\phi_{k}$ span the irreducible two-dimensional space that $|\ell\r|k\r$
is in under the action of $W$. In this space, $W$ implements a Pauli-Y rotation by an angle of
$-2\arccos(E_k / \lambda)$.

Thus, the walk operator $W$ encodes the spectrum of $H$ as a function of eigenphases of $W$
s.t. $\mathrm{spectrum}(H) = \lambda \cos(\arg(\mathrm{spectrum}(W)))$
where $\arg(e^{i\phi}) = \phi$.

#### Parameters
 - `select`: The SELECT lcu gate implementing $\select=\sum_l |l\r\l l|H_l$.
 - `prepare`: The PREPARE lcu gate implementing $\prepare|\vec0> = \sum_l \sqrt{\frac{w_l}{\lambda}} |l\r = |\ell\r$
 - `power`: Constructs $W^n$ by repeatedly decomposing into `power` copies of $W$. Defaults to 1. 

#### References
[Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Figure 1.


In [ ]:
from qualtran.bloqs.qubitization.prepare import BlackBoxPrepare, DummyPrepare
from qualtran.bloqs.qubitization.select_bloq import BlackBoxSelect, DummySelect
from qualtran.bloqs.qubitization.walk import Walk

select = BlackBoxSelect(DummySelect())
prepare = BlackBoxPrepare(DummyPrepare())

bloq = Walk(select=select, prepare=prepare)
show_bloq(bloq)

In [ ]:
walk = Walk(select=select, prepare=prepare)
cwalk = walk.controlled()
show_bloq(cwalk)

In [ ]:
show_bloq(walk.decompose_bloq())

In [ ]:
show_bloq(cwalk.decompose_bloq())

## `Reflect`
Applies reflection around a state prepared by `prepare`

$\def\select{\mathrm{SELECT}} \def\prepare{\mathrm{PREPARE}} \def\r{\rangle} \def\l{\langle}$
Applies $R_s = I - 2|s\r\l s|$ using $R_s = P^†(I - 2|0\r\l 0|)P$ where $P$ prepares the state
along which we want to reflect: $P|0\r = |s\r$.

The reflection operator that adds a $-1$ phase to all states in the subspace spanned by $|s\r$.

#### Parameters
 - `prepare`: The prepare bloq.
 - `cv`: If 0/1, a controlled version of the reflection operator is constructed. Defaults to None, in which case the resulting reflection operator is not controlled. 

#### References
[Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Figure 1.


In [ ]:
from qualtran.bloqs.qubitization.prepare import BlackBoxPrepare, DummyPrepare
from qualtran.bloqs.qubitization.reflect import Reflect

prepare = BlackBoxPrepare(DummyPrepare())
bloq = Reflect(prepare=prepare)
show_bloq(bloq)

In [ ]:
# decomposition using ancilla to perform the reflection
show_bloq(bloq.decompose_bloq())

In [ ]:
# decomposition of the controlled version which uses the control
# line to do the refelction.
show_bloq(bloq.controlled().decompose_bloq())

## Walk with reflect decomposition

We show the decomposition of `Walk` while flat-decomposing the `R[Prep]` bloq as well.

In [ ]:
from qualtran.drawing import (
    get_musical_score_data, draw_musical_score, dump_musical_score)

In [ ]:
# Uncontrolled
walk_flat = walk.decompose_bloq().flatten_once(lambda binst: isinstance(binst.bloq, Reflect))
fig, ax = draw_musical_score(get_musical_score_data(walk_flat))
fig.set_figwidth(8)
ax.axis('off')
fig.tight_layout()

In [ ]:
# Controlled
cwalk_flat = cwalk.decompose_bloq().flatten_once(lambda binst: isinstance(binst.bloq, Reflect))
fig, ax = draw_musical_score(get_musical_score_data(cwalk_flat))
fig.set_figwidth(8.5)
ax.axis('off')
fig.tight_layout()

## Walk with select decomposition

We show the decomposition of `Walk` where we break-out the specific `Select` registers out of `BlackBoxSelect`.

In [ ]:
from qualtran.bloqs.controlled_bloq import ControlledBloq
from qualtran.bloqs.util_bloqs import Partition
from qualtran import LeftDangle
from qualtran.drawing import LineManager

c5 = cwalk_flat.flatten_once(lambda b: isinstance(b.bloq, (ControlledBloq)))

class LM(LineManager):
    def maybe_reserve(self, binst, reg, idx):
        if binst is LeftDangle and reg.name == 'selection':
            self.reserve_n(3, lambda b,r: b.bloq_is(Partition))

fig, ax = draw_musical_score(get_musical_score_data(c5, LM()))
fig.set_figwidth(7.5)
fig.tight_layout()